# Demos: Lecture 15

In [1]:
import pennylane as qml
from pennylane import numpy as np

## Demo 1: Quantum phase estimation

<img src="fig/qpe_full.png" width="800px">


In [2]:
T = np.diag([1, np.exp(1j * np.pi/4)])
U = np.kron(T, np.kron(T, T))

In [3]:
eigvals, eigvecs = np.linalg.eig(U)

In [4]:
target_eigvec = eigvecs[:, -1]
target_eigval = eigvals[-1]

In [5]:
target_eigvec

tensor([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j], requires_grad=True)

In [6]:
target_eigval

(-0.7071067811865474+0.7071067811865477j)

In [7]:
np.angle(target_eigval) / (2 * np.pi) # 001

0.375

In [8]:
estimation_wires = range(3)
target_wires = range(3, 6)

In [9]:
dev = qml.device('default.qubit', wires=6, shots=1)

In [10]:
@qml.qnode(dev)
def quantum_phase_estimation():
    # Prepare uniform sup on estimation wires
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    
    # Prepare eigenstate on target wires
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    
    # Apply controlled rotations
    for control_wire in estimation_wires:
        some_power = 2 ** (len(estimation_wires) - control_wire - 1)
        qml.ControlledQubitUnitary(
            np.linalg.matrix_power(U, some_power),
            control_wires=control_wire,
            wires=target_wires
        )
    
    # Apply inverse QFT to estimation wires
    qml.adjoint(qml.QFT)(wires=estimation_wires)
    
    return qml.sample(wires=estimation_wires)

In [11]:
quantum_phase_estimation()

array([0, 1, 1])

In [12]:
@qml.qnode(dev)
def qpe_from_template():
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    qml.QuantumPhaseEstimation(U, target_wires=target_wires, estimation_wires=estimation_wires)
    return qml.sample(wires=estimation_wires)

In [13]:
qpe_from_template()

array([0, 1, 1])

## Demo 2: Order finding

In [14]:
from lecture15_helpers  import *

In [15]:
N = 7
a = 5

In [16]:
U_Na = get_U_Na(a, N)

num_estimation_qubits = 10
num_target_qubits = int(np.log2(len(U_Na)))

estimation_wires = range(num_estimation_qubits)
target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)

dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)

In [17]:
@qml.qnode(dev)
def find_order():
    # Prepare the target register in some state
    qml.PauliX(wires=target_wires[-1])

    # Run QPE
    qml.QuantumPhaseEstimation(
        U_Na,
        target_wires=target_wires,
        estimation_wires=estimation_wires
    )

    return qml.sample(wires=estimation_wires)

In [18]:
possible_r = []

for _ in range(10):
    sample = find_order()
    phase = fractional_binary_to_float(sample)
    est_r = phase_to_order(phase, N)
    possible_r.append(est_r)

In [19]:
possible_r

[6, 2, 2, 6, 1, 3, 6, 6, 3, 6]

## Demo 3: Shor's algorithm

In [22]:
def run_order_finding(a, N):
    U_Na = get_U_Na(a, N)
    
    num_estimation_qubits = 10
    num_target_qubits = int(np.log2(len(U_Na)))
    
    estimation_wires = range(num_estimation_qubits)
    target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)
    
    dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)
    
    @qml.qnode(dev)
    def find_order():
        # Prepare the target register in some state
        qml.PauliX(wires=target_wires[-1])
    
        # Run QPE
        qml.QuantumPhaseEstimation(
            U_Na,
            target_wires=target_wires,
            estimation_wires=estimation_wires
        )
    
        return qml.sample(wires=estimation_wires)

    sample = find_order()
    phase = fractional_binary_to_float(sample)
    est_r = phase_to_order(phase, N)

    return est_r

<img src="fig/shor-flowchart.jpeg" width="300px">

In [23]:
max_its = 10

def shors_algorithm(N):
    
    for _ in range(max_its):
        # Choose a random a in the range {2, N - 2}
        a = np.random.choice(list(range(2, N - 1)))
        print(f"Chose a={a}")

        # Check if not co-prime (lucky!)
        if not np.gcd(a, N) == 1:
            p = np.gcd(a, N)
            q = N // p
            print("We were lucky")
            return p, q

        for it in range(100):
            # Find the period r
            r = run_order_finding(a, N)
            
            # If r is odd, restart and choose a different a
            if r % 2 == 1:
                continue
    
            # Otherwise, compute x = a ^ (r/2) mod N
            x = a ** (r // 2) % N
            
            if x == 1 % N or x == N - 1:
                continue
            else:
                p = np.gcd(x - 1, N)
                q = np.gcd(x + 1, N)
                
                if p * q == N:
                    print(f"Found solution after {it+1} shots:")
                    print(f"r = {r}, x = {x}")
                    print(f"p = {p}, q = {q}")
                    return p, q

In [28]:
N = 77

p, q = shors_algorithm(N)

Chose a=10
Chose a=30
Chose a=18
Found solution after 10 shots:
r = 30, x = 43
p = 7, q = 11
